In [1]:
# IMPORT LIBRARIES
import pandas as pd, numpy as np, os, gc

# LOAD AND FREQUENCY-ENCODE
FE = ['EngineVersion','AppVersion','AvSigVersion','Census_OSVersion']
# LOAD AND ONE-HOT-ENCODE
OHE = [ 'RtpStateBitfield','IsSxsPassiveMode','DefaultBrowsersIdentifier',
        'AVProductStatesIdentifier','AVProductsInstalled', 'AVProductsEnabled',
        'CountryIdentifier', 'CityIdentifier', 
        'GeoNameIdentifier', 'LocaleEnglishNameIdentifier',
        'Processor', 'OsBuild', 'OsSuite',
        'SmartScreen','Census_MDC2FormFactor',
        'Census_OEMNameIdentifier', 
        'Census_ProcessorCoreCount',
        'Census_ProcessorModelIdentifier', 
        'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName',
        'Census_HasOpticalDiskDrive',
        'Census_TotalPhysicalRAM', 'Census_ChassisTypeName',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches',
        'Census_InternalPrimaryDisplayResolutionHorizontal',
        'Census_InternalPrimaryDisplayResolutionVertical',
        'Census_PowerPlatformRoleName', 'Census_InternalBatteryType',
        'Census_InternalBatteryNumberOfCharges',
        'Census_OSEdition', 'Census_OSInstallLanguageIdentifier',
        'Census_GenuineStateName','Census_ActivationChannel',
        'Census_FirmwareManufacturerIdentifier',
        'Census_IsTouchEnabled', 'Census_IsPenCapable',
        'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer',
        'Wdft_RegionIdentifier']

# LOAD ALL AS CATEGORIES
dtypes = {}
for x in FE+OHE: dtypes[x] = 'category'
dtypes['MachineIdentifier'] = 'str'
dtypes['HasDetections'] = 'int8'

# LOAD CSV FILE
df_train = pd.read_csv('train.csv', usecols=dtypes.keys(), dtype=dtypes, nrows=100000)
print ('Loaded',len(df_train),'rows of TRAIN.CSV!')


# DOWNSAMPLE
sm = 100000
df_train = df_train.sample(sm)
print ('Only using',sm,'rows to train and validate')
x=gc.collect()
print(x)

# unique value in columns
print("frequency encoding...")
for f in FE:
    print(f,df_train[f].nunique())

print("one hot encoding...")
for f in OHE:
    print(f,df_train[f].nunique()) 

Loaded 100000 rows of TRAIN.CSV!
Only using 100000 rows to train and validate
3895
frequency encoding...
EngineVersion 42
AppVersion 80
AvSigVersion 4027
Census_OSVersion 260
one hot encoding...
RtpStateBitfield 6
IsSxsPassiveMode 2
DefaultBrowsersIdentifier 288
AVProductStatesIdentifier 2069
AVProductsInstalled 5
AVProductsEnabled 5
CountryIdentifier 221
CityIdentifier 15812
GeoNameIdentifier 242
LocaleEnglishNameIdentifier 186
Processor 3
OsBuild 43
OsSuite 7
SmartScreen 11
Census_MDC2FormFactor 12
Census_OEMNameIdentifier 860
Census_ProcessorCoreCount 17
Census_ProcessorModelIdentifier 1738
Census_PrimaryDiskTotalCapacity 480
Census_PrimaryDiskTypeName 4
Census_HasOpticalDiskDrive 2
Census_TotalPhysicalRAM 172
Census_ChassisTypeName 30
Census_InternalPrimaryDiagonalDisplaySizeInInches 369
Census_InternalPrimaryDisplayResolutionHorizontal 170
Census_InternalPrimaryDisplayResolutionVertical 197
Census_PowerPlatformRoleName 9
Census_InternalBatteryType 22
Census_InternalBatteryNumberOf

In [2]:
import math

# CHECK FOR NAN
def nan_check(x):
    if isinstance(x,float):
        if math.isnan(x):
            return True
    return False

# FREQUENCY ENCODING
def encode_FE(df,col,verbose=1):
    d = df[col].value_counts(dropna=False)
    
    n = col+"_FE"
    df[n] = df[col].map(d)/d.max()
    
    if verbose==1:
        print('FE encoded',col)
    return [n]

# ONE-HOT-ENCODE ALL CATEGORY VALUES THAT COMPRISE MORE THAN
# "FILTER" PERCENT OF TOTAL DATA AND HAS SIGNIFICANCE GREATER THAN "ZVALUE"
def encode_OHE(df, col, filter, zvalue, tar='HasDetections', m=0.5, verbose=1):
    cv = df[col].value_counts(dropna=False)
    cvd = cv.to_dict()
    vals = len(cv)
    th = filter * len(df)
    sd = zvalue * 0.5/ math.sqrt(th)
    #print(sd)
    n = []; ct = 0; d = {}
    for x in cv.index:
        try:
            if cv[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cv[x])
        except:
            if cvd[x]<th: break
            sd = zvalue * 0.5/ math.sqrt(cvd[x])
        if nan_check(x): r = df[df[col].isna()][tar].mean()
        else: r = df[df[col]==x][tar].mean()
        if abs(r-m)>sd:
            nm = col+'_BE_'+str(x)
            if nan_check(x): df[nm] = (df[col].isna()).astype('int8')
            else: df[nm] = (df[col]==x).astype('int8')
            n.append(nm)
            d[x] = 1
        ct += 1
        if (ct+1)>=vals: break
    if verbose==1:
        print('OHE encoded',col,'- Created',len(d),'booleans')
    return [n,d]

# ONE-HOT-ENCODING from dictionary
def encode_OHE_test(df,col,dt):
    n = []
    for x in dt: 
        n += encode_BE(df,col,x)
    return n

# BOOLEAN ENCODING
def encode_BE(df,col,val):
    n = col+"_BE_"+str(val)
    if nan_check(val):
        df[n] = df[col].isna()
    else:
        df[n] = df[col]==val
    df[n] = df[n].astype('int8')
    return [n]

# importatnt link
# one hot encode vs frequency encode: https://www.datacamp.com/community/tutorials/encoding-methodologies



In [3]:
cols = []; dd = []

# ENCODE NEW
for x in FE:
    out = encode_FE(df_train,x)
    cols += out
for x in OHE:
    tmp = encode_OHE(df_train,x,0.005,5)
#     print("tmp:",tmp)
    cols += tmp[0]; dd.append(tmp[1])
print('Encoded',len(cols),'new variables')
# print("cols:",cols)
# print("dd:",dd)
# REMOVE OLD
for x in FE+OHE:
    del df_train[x]

#df_train.to_csv('my_csv_1L.csv', mode='a', header=True)

print('Removed original',len(FE+OHE),'variables')
x = gc.collect()
print(x)

FE encoded EngineVersion
FE encoded AppVersion
FE encoded AvSigVersion
FE encoded Census_OSVersion
OHE encoded RtpStateBitfield - Created 1 booleans
OHE encoded IsSxsPassiveMode - Created 0 booleans
OHE encoded DefaultBrowsersIdentifier - Created 0 booleans
OHE encoded AVProductStatesIdentifier - Created 6 booleans
OHE encoded AVProductsInstalled - Created 3 booleans
OHE encoded AVProductsEnabled - Created 1 booleans
OHE encoded CountryIdentifier - Created 3 booleans
OHE encoded CityIdentifier - Created 2 booleans
OHE encoded GeoNameIdentifier - Created 2 booleans
OHE encoded LocaleEnglishNameIdentifier - Created 3 booleans
OHE encoded Processor - Created 2 booleans
OHE encoded OsBuild - Created 3 booleans
OHE encoded OsSuite - Created 1 booleans
OHE encoded SmartScreen - Created 3 booleans
OHE encoded Census_MDC2FormFactor - Created 3 booleans
OHE encoded Census_OEMNameIdentifier - Created 2 booleans
OHE encoded Census_ProcessorCoreCount - Created 5 booleans
OHE encoded Census_Process

In [4]:
from keras import callbacks
from sklearn.metrics import roc_auc_score

class printAUC(callbacks.Callback):
    def __init__(self, X_train, y_train):
        super(printAUC, self).__init__()
        self.bestAUC = 0
        self.X_train = X_train
        self.y_train = y_train
        
    def on_epoch_end(self, epoch, logs={}):
        pred = self.model.predict(np.array(self.X_train))
        auc = roc_auc_score(self.y_train, pred)
        print("Train AUC: " + str(auc))
        pred = self.model.predict(self.validation_data[0])
        auc = roc_auc_score(self.validation_data[1], pred)
        print ("Validation AUC: " + str(auc))
        if (self.bestAUC < auc) :
            self.bestAUC = auc
            self.model.save("bestNet.h5", overwrite=True)
        return

Using TensorFlow backend.


In [14]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.callbacks import LearningRateScheduler
from keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

#SPLIT TRAIN AND VALIDATION SET
X_train, X_val, Y_train, Y_val = train_test_split(
    df_train[cols], df_train['HasDetections'], test_size = 0.5)

# BUILD MODEL
def create_model():
    model = Sequential()
    model.add(Dense(100,input_dim=len(cols)))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(100))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=0.01), loss="binary_crossentropy", metrics=["accuracy"])
    return model

annealer = LearningRateScheduler(lambda x: 1e-2 * 0.95 ** x)

model = KerasClassifier(build_fn=create_model)

batch_size = [10, 20, 40, 60, 80, 100]
epochs = [1, 2, 3]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring="accuracy")
grid_result = grid.fit(X_train, Y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# TRAIN MODEL
# model.fit(X_train,Y_train, batch_size=32, epochs = 20, callbacks=[annealer,
#           printAUC(X_train, Y_train)], validation_data = (X_val,Y_val), verbose=2)


Epoch 1/1
33333/33333 [==============================] - 17s 522us/step - loss: 0.6711 - acc: 0.5926
Epoch 1/1
33333/33333 [==============================] - 17s 514us/step - loss: 0.6688 - acc: 0.5883
Epoch 1/1
33334/33334 [==============================] - 18s 531us/step - loss: 0.6706 - acc: 0.5884
Epoch 1/2
33333/33333 [==============================] - 18s 528us/step - loss: 0.6702 - acc: 0.5872
Epoch 2/2
33333/33333 [==============================] - 14s 416us/step - loss: 0.6624 - acc: 0.5941
Epoch 1/2
33333/33333 [==============================] - 18s 543us/step - loss: 0.6702 - acc: 0.5876
Epoch 2/2
33333/33333 [==============================] - 16s 482us/step - loss: 0.6606 - acc: 0.5993
Epoch 1/2
33334/33334 [==============================] - 18s 552us/step - loss: 0.6720 - acc: 0.5868
Epoch 2/2
33334/33334 [==============================] - 16s 474us/step - loss: 0.6622 - acc: 0.5963
Epoch 1/3
33333/33333 [==============================] - 19s 557us/step - loss: 0.6706 - ac

Epoch 1/3
33333/33333 [==============================] - 11s 340us/step - loss: 0.6621 - acc: 0.5969
Epoch 2/3
33333/33333 [==============================] - 4s 129us/step - loss: 0.6501 - acc: 0.6103
Epoch 3/3
33333/33333 [==============================] - 4s 131us/step - loss: 0.6475 - acc: 0.6130
Epoch 1/3
33333/33333 [==============================] - 12s 351us/step - loss: 0.6626 - acc: 0.5992
Epoch 2/3
33333/33333 [==============================] - 4s 130us/step - loss: 0.6475 - acc: 0.6113
Epoch 3/3
33333/33333 [==============================] - 4s 129us/step - loss: 0.6471 - acc: 0.6159
Epoch 1/3
33334/33334 [==============================] - 12s 355us/step - loss: 0.6654 - acc: 0.5915
Epoch 2/3
33334/33334 [==============================] - 4s 131us/step - loss: 0.6513 - acc: 0.6123
Epoch 3/3
33334/33334 [==============================] - 4s 130us/step - loss: 0.6480 - acc: 0.6156
Epoch 1/1
33333/33333 [==============================] - 12s 361us/step - loss: 0.6633 - acc: 0.5

In [7]:
# del df_train
# del X_train, X_val, Y_train, Y_val
x = gc.collect()

# LOAD BEST SAVED NET
from keras.models import load_model
model = load_model('bestNet.h5')

pred = np.zeros((7853253,1))
id = 1
chunksize = 2000000
for df_test in pd.read_csv('test.csv', 
            chunksize = chunksize, usecols=list(dtypes.keys())[0:-1], dtype=dtypes):
    print ('Loaded',len(df_test),'rows of TEST.CSV!')
    # ENCODE TEST
    cols = []
    for x in FE:
        cols += encode_FE(df_test,x,verbose=0)
    for x in range(len(OHE)):
        cols += encode_OHE_test(df_test,OHE[x],dd[x])
    # PREDICT TEST
    end = (id)*chunksize
    if end>7853253: end = 7853253
    pred[(id-1)*chunksize:end] = model.predict(df_test[cols])
    print('  encoded and predicted part',id)
    id += 1
    
    

Loaded 2000000 rows of TEST.CSV!
  encoded and predicted part 1
Loaded 2000000 rows of TEST.CSV!
  encoded and predicted part 2
Loaded 2000000 rows of TEST.CSV!
  encoded and predicted part 3
Loaded 1853253 rows of TEST.CSV!
  encoded and predicted part 4


In [8]:

# SUBMIT TO KAGGLE
df_test = pd.read_csv('test.csv', usecols=['MachineIdentifier'])
df_test['HasDetections'] = pred
df_test.to_csv('submission.csv', index=False)